# Assignment8

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model

# How Much is Your Car Worth?

Data about the retail price of 2005 General Motors cars can be found in `car_data.csv`.

The columns are:

1. Price: suggested retail price of the used 2005 GM car in excellent condition.
2. Mileage: number of miles the car has been driven
3. Make: manufacturer of the car such as Saturn, Pontiac, and Chevrolet
4. Model: specific models for each car manufacturer such as Ion, Vibe, Cavalier
5. Trim (of car): specific type of car model such as SE Sedan 4D, Quad Coupe 2D          
6. Type: body type such as sedan, coupe, etc.      
7. Cylinder: number of cylinders in the engine        
8. Liter: a more specific measure of engine size     
9. Doors: number of doors           
10. Cruise: indicator variable representing whether the car has cruise control (1 = cruise)
11. Sound: indicator variable representing whether the car has upgraded speakers (1 = upgraded)
12. Leather: indicator variable representing whether the car has leather seats (1 = leather)

## Tasks, Part 1

1. Find the linear regression equation for mileage vs price.
2. Chart the original data and the equation on the chart.
3. Find the equation's $R^2$ score (use the `.score` method) to determine whether the
equation is a good fit for this data. (0.8 and greater is considered a strong correlation.)

## Tasks, Part 2

1. Use mileage, cylinders, liters, doors, cruise, sound, and leather to find the linear regression equation.
2. Find the equation's $R^2$ score (use the `.score` method) to determine whether the
equation is a good fit for this data. (0.8 and greater is considered a strong correlation.)
3. Find the combination of the factors that is the best predictor for price.

## Tasks, Hard Mode

1. Research dummy variables in scikit-learn to see how to use the make, model, and body type.
2. Find the best combination of factors to predict price.

In [83]:
df = pd.read_csv("car_data.csv")
df.shape
df.head(5)

,Price,Mileage,Make,Model,Trim,Type,Cylinder,Liter,Doors,Cruise,Sound,Leather
0,17314.103129,8221,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,1
1,17542.036083,9135,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
2,16218.847862,13196,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
3,16336.913140,16342,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,0
4,16339.170324,19832,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,1


In [72]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler

x1 = df[['Mileage']]
y1 = df[['Price']]

x1_train,x1_test,y1_train,y1_test = train_test_split(x1,y1,test_size = 0.3,random_state = 500)
#print(x1_train)
#print(x1_test)

model1 = LinearRegression()

#folds = KFold(n_splits = 5, shuffle = True, random_state = 10)
#pred1 = cross_val_predict(model1,x1,y1,cv=folds)

#R2_1 = r2_score(y1,pred1)
#print(R2_1)
model1 = LinearRegression()
model1.fit(x1_train,y1_train)
pred1 = model1.predict(x1_test)
model1.score(x1_test,y1_test)

0.0359551936072533

# Task Part 2

In [76]:
x2 = df[['Mileage','Cylinder','Liter','Doors','Cruise','Sound','Leather']]
y2 = df[['Price']]

x2_train,x2_test,y2_train,y2_test = train_test_split(x2,y2,test_size = 0.3,random_state = 500)

Min_Max_Scaler = MinMaxScaler(feature_range = (0,1))
tr_xtrain = Min_Max_Scaler.fit_transform(x2_train)
tr_xtest = Min_Max_Scaler.transform(x2_test)

In [80]:
model2 = LinearRegression()
support = []
r2Score = []

for features in range(1,x2.shape[1]+1):
    rfe = RFE(model2, n_features_to_select=features)
    rfe.fit(tr_xtrain,y2_train)
    support.append(rfe.support_)
    model2.fit(tr_xtrain[:,rfe.support_],y2_train)
    predicted2 = model2.predict(tr_xtest[:,rfe.support_])
    r2Score.append(model2.score(tr_xtest[:,rfe.support_],y2_test))

In [82]:
print("The model with best R2 score is with : ", np.array(r2Score).argmax()+1 ,"columns, which is : ",np.array(r2Score).max())
print("The best selections of columns is : ",x2.columns[support[np.array(r2Score).argmax()]],"columns")

The model with best R2 score is with :  6 columns, which is :  0.4480686721398893
The best selections of columns is :  Index(['Mileage', 'Cylinder', 'Doors', 'Cruise', 'Sound', 'Leather'], dtype='object') columns


# Task Part 3

In [98]:
x3 = df[['Mileage','Make','Model','Type','Cylinder','Liter','Doors','Cruise','Sound','Leather']]
y3 = df[['Price']]

encoded_data = pd.get_dummies(x3[['Make','Model','Type']], columns = ['Make','Model','Type'])
encoded_data

,Make_Buick,Make_Cadillac,Make_Chevrolet,Make_Pontiac,Make_SAAB,Make_Saturn,Model_9-2X AWD,Model_9_3,Model_9_3 HO,Model_9_5,...,Model_STS-V6,Model_STS-V8,Model_Sunfire,Model_Vibe,Model_XLR-V8,Type_Convertible,Type_Coupe,Type_Hatchback,Type_Sedan,Type_Wagon
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
800,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
801,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
802,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [100]:
x3.drop(['Make','Model','Type'],axis = 1,inplace = True)
new_features = x3.join(encoded_data)
new_features.head(5)

,Mileage,Cylinder,Liter,Doors,Cruise,Sound,Leather,Make_Buick,Make_Cadillac,Make_Chevrolet,...,Model_STS-V6,Model_STS-V8,Model_Sunfire,Model_Vibe,Model_XLR-V8,Type_Convertible,Type_Coupe,Type_Hatchback,Type_Sedan,Type_Wagon
0,8221,6,3.1,4,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,9135,6,3.1,4,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,13196,6,3.1,4,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,16342,6,3.1,4,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,19832,6,3.1,4,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [101]:
x3_train,x3_test,y3_train,y3_test = train_test_split(new_features,y3,test_size = 0.3,random_state = 500)

Min_Max_Scaler = MinMaxScaler(feature_range = (0,1))
tr_xtrain3 = Min_Max_Scaler.fit_transform(x3_train)
tr_xtest3 = Min_Max_Scaler.transform(x3_test)

In [104]:
model3 = LinearRegression()
support3 = []
r2Score3 = []

for features in range(1,x3_train.shape[1]+1):
    rfe = RFE(model3, n_features_to_select=features)
    rfe.fit(tr_xtrain3,y3_train)
    support3.append(rfe.support_)
    model3.fit(tr_xtrain3[:,rfe.support_],y3_train)
    predicted3 = model3.predict(tr_xtest3[:,rfe.support_])
    r2Score3.append(model3.score(tr_xtest3[:,rfe.support_],y3_test))

In [106]:
print("The model with best R2 score is with : ", np.array(r2Score3).argmax()+1 ,"columns, which is : ",np.array(r2Score3).max())
print("The best selections of columns is : ",x3_train.columns[support3[np.array(r2Score3).argmax()]],"columns")

The model with best R2 score is with :  50 columns, which is :  0.9818952801139329
The best selections of columns is :  Index(['Mileage', 'Cylinder', 'Liter', 'Doors', 'Cruise', 'Sound', 'Leather',
       'Make_Buick', 'Make_Cadillac', 'Make_Chevrolet', 'Make_Pontiac',
       'Make_SAAB', 'Make_Saturn', 'Model_9-2X AWD', 'Model_9_3',
       'Model_9_3 HO', 'Model_9_5', 'Model_9_5 HO', 'Model_AVEO',
       'Model_Bonneville', 'Model_CST-V', 'Model_CTS', 'Model_Cavalier',
       'Model_Century', 'Model_Classic', 'Model_Cobalt', 'Model_Corvette',
       'Model_Deville', 'Model_G6', 'Model_GTO', 'Model_Grand Am',
       'Model_Grand Prix', 'Model_Impala', 'Model_Ion', 'Model_L Series',
       'Model_Lacrosse', 'Model_Lesabre', 'Model_Malibu', 'Model_Monte Carlo',
       'Model_Park Avenue', 'Model_STS-V6', 'Model_STS-V8', 'Model_Sunfire',
       'Model_Vibe', 'Model_XLR-V8', 'Type_Convertible', 'Type_Coupe',
       'Type_Hatchback', 'Type_Sedan', 'Type_Wagon'],
      dtype='object') column